In [1]:
!pip install -q keras-core jax jaxlib 


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] = "jax"

import keras_core as keras

Using JAX backend.


# Test with MNIST ConvNET

## Load Dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print(f'[STEP 01] | X_train shape is {x_train.shape}, X_test shape is {x_test.shape}')

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print(f'[STEP 02] | X_train shape is {x_train.shape}, X_test shape is {x_test.shape}')

print(f"Train Samples : {x_train.shape[0]}, Test Samples : {x_test.shape[0]}")

[STEP 01] | X_train shape is (60000, 28, 28), X_test shape is (10000, 28, 28)
[STEP 02] | X_train shape is (60000, 28, 28, 1), X_test shape is (10000, 28, 28, 1)
Train Samples : 60000, Test Samples : 10000


## Model Parameters

In [4]:
num_classes = 10
input_shape = x_train.shape[1:]

kernel1 = (2,2)
kernel2 = (3,3)
kernel3 = (5,5)

act1 = 'relu'
act2 = 'gelu'
act3 = 'softmax'

dropout = 0.5

lr = 1e-3

In [5]:
model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=kernel2, activation=act1),
        keras.layers.Conv2D(64, kernel_size=kernel2, activation=act2),
        keras.layers.MaxPooling2D(pool_size=kernel1),
        keras.layers.Conv2D(128, kernel_size=kernel2, activation=act1),
        keras.layers.Conv2D(128, kernel_size=kernel3, activation=act2),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(num_classes, activation=act3)
    ]
)

model.summary()

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)        │        640 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)        │     36,928 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)       │     73,856 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 6, 128)         │    409,728 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling2d        │ (None, 128)               │          0 │
│ (GlobalAveragePooling2D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 10)                │      1,290 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 522,442 (1.99 MB)

 Trainable params: 522,442 (1.99 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.AdamW(learning_rate=lr),
    metrics = [
        keras.metrics.SparseCategoricalAccuracy(name='acc'),
    ],
)

In [7]:
batch_size = 2048
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath='model_epoch{epoch}.keras'), # 여기 filepath 이렇게 작성하는 게 맞나?
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=int(epochs*0.1)),
]

model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 123s 5s/step - acc: 0.2625 - loss: 2.0596 - val_acc: 0.8166 - val_loss: 0.7905
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 114s 5s/step - acc: 0.7165 - loss: 0.8700 - val_acc: 0.9180 - val_loss: 0.3052
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 119s 5s/step - acc: 0.8599 - loss: 0.4619 - val_acc: 0.9514 - val_loss: 0.1883
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step - acc: 0.9101 - loss: 0.3027 - val_acc: 0.9659 - val_loss: 0.1316
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 108s 4s/step - acc: 0.9339 - loss: 0.2262 - val_acc: 0.9748 - val_loss: 0.1040
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step - acc: 0.9466 - loss: 0.1819 - val_acc: 0.9726 - val_loss: 0.1022
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - acc: 0.9537 - loss: 0.1600 - val_acc: 0.9763 - val_loss: 0.0880
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step - acc: 0.9608 - loss: 0.1344 - val_acc: 0.9819 - val_loss: 0.0719
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step - acc: 0.9652

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)

In [10]:
model = keras.saving.load_model("model_epoch20.keras")

In [11]:
predictions = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step


# Cross-Framework with custom components

## Define custom components

In [30]:
class newDense(keras.layers.Layer):
    def __init__(self, units, activation=None, name=None):
        super().__init__(name=name)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=(input_dim, self.units),
            initializer=keras.initializers.GlorotNormal(),
            name='kernel',
            trainable=True,
        )
        
        self.b = self.add_weight(
            shape=(self.units,),
            initializer=keras.initializers.Zeros(),
            name='bias',
            trainable=True,
        )
        
    def call(self, inputs):
        x = keras.ops.matmul(inputs, self.w) + self.b
        return self.activation(x)

In [31]:
class newDropout(keras.layers.Layer):
    def __init__(self, rate, name=None):
        super().__init__(name=name)
        self.rate = rate
        self.seed_generator = keras.random.SeedGenerator(2023)
        
    def call(self, inputs):
        return keras.random.dropout(inputs, self.rate, seed=self.seed_generator)

In [32]:
class newModel(keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_base = keras.Sequential(
            [
                keras.layers.Input(shape=input_shape),
                keras.layers.Conv2D(64, kernel_size=kernel2, activation=act1),
                keras.layers.Conv2D(64, kernel_size=kernel2, activation=act2),
                keras.layers.MaxPooling2D(pool_size=kernel1),
                keras.layers.Conv2D(128, kernel_size=kernel2, activation=act1),
                keras.layers.Conv2D(128, kernel_size=kernel3, activation=act2),
                keras.layers.GlobalAveragePooling2D(),
            ]
        )
        
        self.dp = newDropout(0.5)
        self.dense = newDense(num_classes, activation=act3)
        
    def call(self, x):
        x = self.conv_base(x)
        x = self.dp(x)
        return self.dense(x)

In [33]:
model2 = newModel(num_classes=10)
model2.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
               optimizer=keras.optimizers.AdamW(learning_rate=lr),
               metrics=[
                   keras.metrics.SparseCategoricalAccuracy(name='acc'),
               ])

model2.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=10,
    validation_split=0.15
)

Epoch 1/10


25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - acc: 0.2673 - loss: 2.0482 - val_acc: 0.7207 - val_loss: 0.8716
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - acc: 0.7560 - loss: 0.7615 - val_acc: 0.8771 - val_loss: 0.3966
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - acc: 0.8869 - loss: 0.3828 - val_acc: 0.9280 - val_loss: 0.2547
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - acc: 0.9213 - loss: 0.2630 - val_acc: 0.9369 - val_loss: 0.2110
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - acc: 0.9393 - loss: 0.2082 - val_acc: 0.9514 - val_loss: 0.1676
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - acc: 0.9488 - loss: 0.1725 - val_acc: 0.9580 - val_loss: 0.1530
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 106s 4s/step - acc: 0.9552 - loss: 0.1476 - val_acc: 0.9678 - val_loss: 0.1200
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 108s 4s/step - acc: 0.9618 - loss: 0.1304 - val_acc: 0.9687 - val_loss: 0.1228
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 108s 4s/step - acc: 0.9629 - loss: 0.

## torch

In [35]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 48.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.1 MB/s eta 0:00:0000:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 6.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 20.8 M

In [52]:
import torch

train_torch_ds = torch.utils.data.TensorDataset(
    torch.from_numpy(x_train), torch.from_numpy(y_train)
)
val_torch_ds = torch.utils.data.TensorDataset(
    torch.from_numpy(x_test), torch.from_numpy(y_test)
)

train_dl = torch.utils.data.DataLoader(
    train_torch_ds, batch_size=batch_size, shuffle=True
)
val_dl = torch.utils.data.DataLoader(
    val_torch_ds, batch_size=batch_size, shuffle=False
)

model3 = newModel(num_classes=10)

model3.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
               optimizer=keras.optimizers.AdamW(learning_rate=lr),
               metrics=[
                   keras.metrics.SparseCategoricalAccuracy(name='acc'),
               ])

model3.fit(
    train_dl,
    validation_data=val_dl,
    epochs=3
)

Epoch 1/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 127s 4s/step - acc: 0.2984 - loss: 1.9927 - val_acc: 0.7699 - val_loss: 0.7152
Epoch 2/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 123s 4s/step - acc: 0.8109 - loss: 0.6057 - val_acc: 0.8902 - val_loss: 0.3470
Epoch 3/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 122s 4s/step - acc: 0.9041 - loss: 0.3196 - val_acc: 0.9369 - val_loss: 0.2064


## tensorflow

In [72]:
!pip install tensorflow==2.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 36.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.

In [65]:
!pip uninstall numpy -y

Found existing installation: numpy 1.26.1
Uninstalling numpy-1.26.1:
  Successfully uninstalled numpy-1.26.1


In [66]:
!pip install numpy==1.23.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 49.3 MB/s eta 0:00:0000:0100:01


In [67]:
!pip show numpy

Name: numpy
Version: 1.23.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/wfs/.pyenv/versions/3.11.2/lib/python3.11/site-packages
Requires: 
Required-by: h5py, jax, jaxlib, keras-core, ml-dtypes, opt-einsum, scipy, tensorboard, tensorflow


In [73]:
import tensorflow as tf

train_tf_ds = (
    tf.data.Datasets.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
test_tf_ds = (
    tf.data.Datasets.from_tensor_slices((x_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

model4 = newModel(num_classes=10)

model4.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
               optimizer=keras.optimizers.AdamW(learning_rate=lr),
               metrics=[
                   keras.metrics.SparseCategoricalAccuracy(name='acc'),
               ])

model4.fit(
    train_tf_ds, 
    validation_data=test_tf_ds,
    epochs=3
)

ImportError: /home/wfs/.pyenv/versions/3.11.2/lib/python3.11/site-packages/tensorflow/python/client/_pywrap_tf_session.so: undefined symbol: _ZN10tensorflow11SetFullTypeEP8TF_GraphP12TF_OperationRKNS_11FullTypeDefE, version tensorflow

In [ ]:
# python version, numpy, tensorflow가 계속 버전 충돌을 일으킴
# 어지간하면 tensorflow는 배제하는게 맞을듯